## 1. INITIATION & READING DATA

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_colwidth', -1)

import seaborn as sns
sns.set_style('whitegrid')

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')


from sklearn import feature_selection
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

import lightgbm as lgb

print('** M5 - FORECASTING - ACCURACY **')
print("-"*25)

import gc
import os
for x in os.listdir("input/"):
    print(x)
print("-"*25)

print ("Initiation is DONE !!")
print("-"*25)

** M5 - FORECASTING - ACCURACY **
-------------------------
calendar.csv
model_raw_lgbm1_regression_gbdt_0.3_0.3_0.3.txt
sales_train_evaluation.csv
sales_train_validation.csv
sample_submission.csv
sell_prices.csv
-------------------------
Initiation is DONE !!
-------------------------


In [2]:
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)

In [3]:
##### SOME USEFULL FUNCTIONS

## TIMER
import time
from contextlib import contextmanager
@contextmanager
def timer():
    t0 = time.time()
    yield
    t1 = time.time()
    print( 'Runtime: ' + str(round(t1-t0,2)) + ' sn' )

In [4]:
path = 'input/'

with timer():
    calendar   = pd.read_csv( f'{path}calendar.csv')
#    train_v    = pd.read_csv( f'{path}sales_train_validation.csv')
    train    = pd.read_csv( f'{path}sales_train_evaluation.csv')
    prices     = pd.read_csv( f'{path}sell_prices.csv')
    submission = pd.read_csv( f'{path}sample_submission.csv')

print('Reading Data is DONE !!')
print("-"*25)

print( f'calendar   shape :  {calendar.shape}    ')
#print( f'train_v    shape :  {train_v.shape}     ')
print( f'train    shape :  {train.shape}     ')
print( f'prices     shape :  {prices.shape}      ')
print( f'submission shape :  {submission.shape}  ')

Runtime: 5.51 sn
Reading Data is DONE !!
-------------------------
calendar   shape :  (1969, 14)    
train    shape :  (30490, 1947)     
prices     shape :  (6841121, 4)      
submission shape :  (60980, 29)  


## 2. BASIC PREPROCESSING

### 2.1. TRAIN DATA PREPARATION

In [5]:
print(train.shape)
print('-'*25)
train.head()

(30490, 1947)
-------------------------


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299,d_300,d_301,d_302,d_303,d_304,d_305,d_306,d_307,d_308,d_309,d_310,d_311,d_312,d_313,d_314,d_315,d_316,d_317,d_318,d_319,d_320,d_321,d_322,d_323,d_324,d_325,d_326,d_327,d_328,d_329,d_330,d_331,d_332,d_333,d_334,d_335,d_336,d_337,d_338,d_339,d_340,d_341,d_342,d_343,d_344,d_345,d_346,d_347,d_348,d_349,d_350,d_351,d_352,d_353,d_354,d_355,d_356,d_357,d_358,d_359,d_360,d_361,d_362,d_363,d_364,d_365,d_366,d_367,d_368,d_369,d_370,d_371,d_372,d_373,d_374,d_375,d_376,d_377,d_378,d_379,d_380,d_381,d_382,d_383,d_384,d_385,d_386,d_387,d_388,d_389,d_390,d_391,d_392,d_393,d_394,d_395,d_396,d_397,d_398,d_399,d_400,d_401,d_402,d_403,d_404,d_405,d_406,d_407,d_408,d_409,d_410,d_411,d_412,d_413,d_414,d_415,d_416,d_417,d_418,d_419,d_420,d_421,d_422,d_423,d_424,d_425,d_426,d_427,d_428,d_429,d_430,d_431,d_432,d_433,d_434,d_435,d_436,d_437,d_438,d_439,d_440,d_441,d_442,d_443,d_444,d_445,d_446,d_447,d_448,d_449,d_450,d_451,d_452,d_453,d_454,d_455,d_456,d_457,d_458,d_459,d_460,d_461,d_462,d_463,d_464,d_465,d_466,d_467,d_468,d_469,d_470,d_471,d_472,d_473,d_474,d_475,d_476,d_477,d_478,d_479,d_480,d_481,d_482,d_483,d_484,d_485,d_486,d_487,d_488,d_489,d_490,d_491,d_492,d_493,...,d_1443,d_1444,d_1445,d_1446,d_1447,d_1448,d_1449,d_1450,d_1451,d_1452,d_1453,d_1454,d_1455,d_1456,d_1457,d_1458,d_1459,d_1460,d_1461,d_1462,d_1463,d_1464,d_1465,d_1466,d_1467,d_1468,d_1469,d_1470,d_1471,d_1472,d_1473,d_1474,d_1475,d_1476,d_1477,d_1478,d_1479,d_1480,d_1481,d_1482,d_1483,d_1484,d_1485,d_1486,d_1487,d_1488,d_1489,d_1490,d_1491,d_1492,d_1493,d_1494,d_1495,d_1496,d_1497,d_1498,d_1499,d_1500,d_1501,d_1502,d_1503,d_1504,d_1505,d_1506,d_1507,d_1508,d_1509,d_1510,d_1511,d_1512,d_1513,d_1514,d_1515,d_1516,d_1517,d_1518,d_1519,d_1520,d_1521,d_1522,d_1523,d_1524,d_1525,d_1526,d_1527,d_1528,d_1529,d_1530,d_1531,d_1532,d_1533,d_1534,d_1535,d_1536,d_1537,d_1538,d_1539,d_1540,d_1541,d_1542,d_1543,d_1544,d_1545,d_1546,d_1547,d_1548,d_1549,d_1550,d_1551,d_1552,d_1553,d_1554,d_1555,d_1556,d_1557,d_1558,d_1559,d_1560,d_1561,d_1562,d_1563,d_1564,d_1565,d_1566,d_1567,d_1568,d_1569,d_1570,d_1571,d_1572,d_1573,d_1574,d_1575,d_1576,d_1577,d_1578,d_1579,d_1580,d_1581,d_1582,d_1583,d_1584,d_1585,d_1586,d_1587,d_1588,d_1589,d_1590,d_1591,d_1592,d_1593,d_1594,d_1595,d_1596,d_1597,d_1598,d_1599,d_

In [6]:
train['id'] = train['item_id'] + '_' + train['store_id']
print(train.shape)
print('-'*25)
train.head()

(30490, 1947)
-------------------------


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299,d_300,d_301,d_302,d_303,d_304,d_305,d_306,d_307,d_308,d_309,d_310,d_311,d_312,d_313,d_314,d_315,d_316,d_317,d_318,d_319,d_320,d_321,d_322,d_323,d_324,d_325,d_326,d_327,d_328,d_329,d_330,d_331,d_332,d_333,d_334,d_335,d_336,d_337,d_338,d_339,d_340,d_341,d_342,d_343,d_344,d_345,d_346,d_347,d_348,d_349,d_350,d_351,d_352,d_353,d_354,d_355,d_356,d_357,d_358,d_359,d_360,d_361,d_362,d_363,d_364,d_365,d_366,d_367,d_368,d_369,d_370,d_371,d_372,d_373,d_374,d_375,d_376,d_377,d_378,d_379,d_380,d_381,d_382,d_383,d_384,d_385,d_386,d_387,d_388,d_389,d_390,d_391,d_392,d_393,d_394,d_395,d_396,d_397,d_398,d_399,d_400,d_401,d_402,d_403,d_404,d_405,d_406,d_407,d_408,d_409,d_410,d_411,d_412,d_413,d_414,d_415,d_416,d_417,d_418,d_419,d_420,d_421,d_422,d_423,d_424,d_425,d_426,d_427,d_428,d_429,d_430,d_431,d_432,d_433,d_434,d_435,d_436,d_437,d_438,d_439,d_440,d_441,d_442,d_443,d_444,d_445,d_446,d_447,d_448,d_449,d_450,d_451,d_452,d_453,d_454,d_455,d_456,d_457,d_458,d_459,d_460,d_461,d_462,d_463,d_464,d_465,d_466,d_467,d_468,d_469,d_470,d_471,d_472,d_473,d_474,d_475,d_476,d_477,d_478,d_479,d_480,d_481,d_482,d_483,d_484,d_485,d_486,d_487,d_488,d_489,d_490,d_491,d_492,d_493,...,d_1443,d_1444,d_1445,d_1446,d_1447,d_1448,d_1449,d_1450,d_1451,d_1452,d_1453,d_1454,d_1455,d_1456,d_1457,d_1458,d_1459,d_1460,d_1461,d_1462,d_1463,d_1464,d_1465,d_1466,d_1467,d_1468,d_1469,d_1470,d_1471,d_1472,d_1473,d_1474,d_1475,d_1476,d_1477,d_1478,d_1479,d_1480,d_1481,d_1482,d_1483,d_1484,d_1485,d_1486,d_1487,d_1488,d_1489,d_1490,d_1491,d_1492,d_1493,d_1494,d_1495,d_1496,d_1497,d_1498,d_1499,d_1500,d_1501,d_1502,d_1503,d_1504,d_1505,d_1506,d_1507,d_1508,d_1509,d_1510,d_1511,d_1512,d_1513,d_1514,d_1515,d_1516,d_1517,d_1518,d_1519,d_1520,d_1521,d_1522,d_1523,d_1524,d_1525,d_1526,d_1527,d_1528,d_1529,d_1530,d_1531,d_1532,d_1533,d_1534,d_1535,d_1536,d_1537,d_1538,d_1539,d_1540,d_1541,d_1542,d_1543,d_1544,d_1545,d_1546,d_1547,d_1548,d_1549,d_1550,d_1551,d_1552,d_1553,d_1554,d_1555,d_1556,d_1557,d_1558,d_1559,d_1560,d_1561,d_1562,d_1563,d_1564,d_1565,d_1566,d_1567,d_1568,d_1569,d_1570,d_1571,d_1572,d_1573,d_1574,d_1575,d_1576,d_1577,d_1578,d_1579,d_1580,d_1581,d_1582,d_1583,d_1584,d_1585,d_1586,d_1587,d_1588,d_1589,d_1590,d_1591,d_1592,d_1593,d_1594,d_1595,d_1596,d_1597,d_1598,d_1599,d_

In [7]:
# Making data usable in data
with timer():
    train = pd.melt( train,
                     id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                     value_vars = [col for col in train.columns if col.startswith("d_")],
                     var_name = "d",
                     value_name = "sales")
with timer():
    train['d']     = train['d'].str.extract('(\d+)').astype('int16')
    train['sales'] = train['sales'].astype('int16')


with timer():
    train.sort_values(['id', 'd'], inplace=True)


print(train.shape)
print('-'*25)
train.tail()

Runtime: 8.99 sn
Runtime: 94.79 sn
Runtime: 42.97 sn
(59181090, 8)
-------------------------


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
59057692,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1937,0
59088182,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1938,0
59118672,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1939,0
59149162,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1940,0
59179652,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1941,0


In [8]:
# Adding the 28 days we try to forecast
with timer():
    tmp = train.groupby('id', as_index=False ).tail(28)
    tmp['d'] = tmp['d'] + 28
    tmp['sales'] = np.nan
    print(tmp.shape)

with timer():
    train = pd.concat([train,tmp], axis=0, sort=False, ignore_index=True)

del tmp; gc.collect()

with timer():
    train.sort_values(['id', 'd'], inplace=True)
    train.reset_index(drop=True, inplace=True)
    
print(train.shape)
print('-'*25)
train.tail()

(853720, 8)
Runtime: 5.28 sn
Runtime: 1.68 sn
Runtime: 19.12 sn
(60034810, 8)
-------------------------


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
60034805,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1965,NaN
60034806,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1966,NaN
60034807,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1967,NaN
60034808,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1968,NaN
60034809,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1969,NaN


### 2.2. CALENDAR DATA PREPARATION

In [9]:
print(calendar.shape)
print('-'*25)
calendar.tail()

(1969, 14)
-------------------------


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


In [10]:
with timer():
    calendar['d'] = calendar['d'].str.extract('(\d+)').astype('int16')
    
    for col in ['wday', 'month']:
        calendar[col] = calendar[col].astype('int8')
    
    for col in ['wm_yr_wk', 'year']:
        calendar[col] = calendar[col].astype('int16')
    
    calendar.drop(['date', 'weekday'], axis=1, inplace=True)
        
print(calendar.shape)
print('-'*25)
calendar.tail()

Runtime: 0.01 sn
(1969, 12)
-------------------------


,wm_yr_wk,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1964,11620,5,6,2016,1965,NaN,NaN,NaN,NaN,0,1,1
1965,11620,6,6,2016,1966,NaN,NaN,NaN,NaN,0,0,0
1966,11620,7,6,2016,1967,NaN,NaN,NaN,NaN,0,0,0
1967,11621,1,6,2016,1968,NaN,NaN,NaN,NaN,0,0,0
1968,11621,2,6,2016,1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


In [11]:
## DO NOT FORGET TO ADD SNAP DATA
snap = calendar[['d', 'snap_CA', 'snap_TX', 'snap_WI']]

snap = pd.melt( snap,
                     id_vars = ['d'],
                     value_vars = [col for col in snap.columns if col.startswith("snap_")],
                     var_name = "state_id",
                     value_name = "is_snap")

snap['state_id'] = snap['state_id'].str[5:].astype('category')
snap['is_snap'] = snap['is_snap'].astype('int8')

print(snap.shape)
print('-'*25)
snap.tail()

(5907, 3)
-------------------------


,d,state_id,is_snap
5902,1965,WI,1
5903,1966,WI,0
5904,1967,WI,0
5905,1968,WI,0
5906,1969,WI,0


In [12]:
calendar.drop(['snap_CA', 'snap_TX', 'snap_WI'], axis=1, inplace=True)

In [13]:
tmp = pd.get_dummies(calendar['event_name_1'], dtype ='int8' )
calendar = pd.concat([calendar, tmp], axis=1)

tmp = pd.get_dummies(calendar['event_type_1'], dtype ='int8' )
calendar = pd.concat([calendar, tmp], axis=1)


tmp = pd.get_dummies(calendar['event_name_2'], prefix = '2', dtype ='int8' )
calendar = pd.concat([calendar, tmp], axis=1)

tmp = pd.get_dummies(calendar['event_type_2'], prefix = '2', dtype ='int8' )
calendar = pd.concat([calendar, tmp], axis=1)

calendar.drop(['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], axis=1, inplace=True)
del tmp; gc.collect()

print(calendar.shape)
print('-'*25)
calendar.tail(5)

(1969, 45)
-------------------------


,wm_yr_wk,wday,month,year,d,Chanukah End,Christmas,Cinco De Mayo,ColumbusDay,Easter,Eid al-Fitr,EidAlAdha,Father's day,Halloween,IndependenceDay,LaborDay,LentStart,LentWeek2,MartinLutherKingDay,MemorialDay,Mother's day,NBAFinalsEnd,NBAFinalsStart,NewYear,OrthodoxChristmas,OrthodoxEaster,Pesach End,PresidentsDay,Purim End,Ramadan starts,StPatricksDay,SuperBowl,Thanksgiving,ValentinesDay,VeteransDay,Cultural,National,Religious,Sporting,2_Cinco De Mayo,2_Easter,2_Father's day,2_OrthodoxEaster,2_Cultural,2_Religious
1964,11620,5,6,2016,1965,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1965,11620,6,6,2016,1966,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1966,11620,7,6,2016,1967,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1967,11621,1,6,2016,1968,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1968,11621,2,6,2016,1969,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0


In [14]:
calendar['Cinco De Mayo']    = calendar['Cinco De Mayo']   + calendar['2_Cinco De Mayo']
calendar['Easter']           = calendar['Easter']          + calendar['2_Easter']
calendar['Father\'s day']    = calendar['Father\'s day']   + calendar['2_Father\'s day']
calendar['OrthodoxEaster']   = calendar['OrthodoxEaster']  + calendar['2_OrthodoxEaster']
calendar['Cultural']         = calendar['Cultural']        + calendar['2_Cultural']
calendar['Religious']        = calendar['Religious']       + calendar['2_Religious']

calendar.drop([col for col in calendar.columns if col.startswith("2_")], axis=1, inplace=True)

print(calendar.shape)
print('-'*25)
calendar.tail(5)

(1969, 39)
-------------------------


,wm_yr_wk,wday,month,year,d,Chanukah End,Christmas,Cinco De Mayo,ColumbusDay,Easter,Eid al-Fitr,EidAlAdha,Father's day,Halloween,IndependenceDay,LaborDay,LentStart,LentWeek2,MartinLutherKingDay,MemorialDay,Mother's day,NBAFinalsEnd,NBAFinalsStart,NewYear,OrthodoxChristmas,OrthodoxEaster,Pesach End,PresidentsDay,Purim End,Ramadan starts,StPatricksDay,SuperBowl,Thanksgiving,ValentinesDay,VeteransDay,Cultural,National,Religious,Sporting
1964,11620,5,6,2016,1965,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1965,11620,6,6,2016,1966,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1966,11620,7,6,2016,1967,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1967,11621,1,6,2016,1968,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1968,11621,2,6,2016,1969,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [15]:
calendar.loc[ (calendar['d'] >= 185 )  & ( calendar['d'] < 185 + 29 ), 'Ramadan starts' ]  = 1
calendar.loc[ (calendar['d'] >= 539 )  & ( calendar['d'] < 539 + 29 ), 'Ramadan starts' ]  = 1
calendar.loc[ (calendar['d'] >= 893 )  & ( calendar['d'] < 893 + 29 ), 'Ramadan starts' ]  = 1
calendar.loc[ (calendar['d'] >= 1248)  & ( calendar['d'] < 1248 + 29), 'Ramadan starts' ]  = 1
calendar.loc[ (calendar['d'] >= 1602)  & ( calendar['d'] < 1602 + 29), 'Ramadan starts' ]  = 1
calendar.loc[ (calendar['d'] >= 1957)                                , 'Ramadan starts' ]  = 1

calendar.loc[ (calendar['d'] >= 185 )  & ( calendar['d'] < 185 + 29 ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 539 )  & ( calendar['d'] < 539 + 29 ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 893 )  & ( calendar['d'] < 893 + 29 ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1248)  & ( calendar['d'] < 1248 + 29), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1602)  & ( calendar['d'] < 1602 + 29), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1957)                                , 'Religious' ]  = 1


In [16]:
calendar.loc[ (calendar['d'] >= 215 - 1  )  & ( calendar['d'] <= 215 + 1  ), 'Eid al-Fitr' ]  = 1
calendar.loc[ (calendar['d'] >= 569 - 1  )  & ( calendar['d'] <= 569 + 1  ), 'Eid al-Fitr' ]  = 1
calendar.loc[ (calendar['d'] >= 923 - 1  )  & ( calendar['d'] <= 923 + 1  ), 'Eid al-Fitr' ]  = 1
calendar.loc[ (calendar['d'] >= 1278 - 1 )  & ( calendar['d'] <= 1278 + 1 ), 'Eid al-Fitr' ]  = 1
calendar.loc[ (calendar['d'] >= 1632 - 1 )  & ( calendar['d'] <= 1632 + 1 ), 'Eid al-Fitr' ]  = 1

calendar.loc[ (calendar['d'] >= 215 - 1  )  & ( calendar['d'] <= 215 + 1  ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 569 - 1  )  & ( calendar['d'] <= 569 + 1  ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 923 - 1  )  & ( calendar['d'] <= 923 + 1  ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1278 - 1 )  & ( calendar['d'] <= 1278 + 1 ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1632 - 1 )  & ( calendar['d'] <= 1632 + 1 ), 'Religious' ]  = 1

In [17]:
calendar.loc[ (calendar['d'] >= 283   )  & ( calendar['d'] <= 283 + 3  ), 'EidAlAdha' ]  = 1
calendar.loc[ (calendar['d'] >= 637   )  & ( calendar['d'] <= 637 + 3  ), 'EidAlAdha' ]  = 1
calendar.loc[ (calendar['d'] >= 991   )  & ( calendar['d'] <= 991 + 3  ), 'EidAlAdha' ]  = 1
calendar.loc[ (calendar['d'] >= 1345  )  & ( calendar['d'] <= 1345 + 3 ), 'EidAlAdha' ]  = 1
calendar.loc[ (calendar['d'] >= 1700  )  & ( calendar['d'] <= 1700 + 3 ), 'EidAlAdha' ]  = 1

calendar.loc[ (calendar['d'] >= 283   )  & ( calendar['d'] <= 283 + 3  ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 637   )  & ( calendar['d'] <= 637 + 3  ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 991   )  & ( calendar['d'] <= 991 + 3  ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1345  )  & ( calendar['d'] <= 1345 + 3 ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1700  )  & ( calendar['d'] <= 1700 + 3 ), 'Religious' ]  = 1

In [18]:
calendar.loc[ (calendar['d'] >= 123  )  & ( calendar['d'] <= 135  ), 'NBAFinalsStart' ]  = 1
calendar.loc[ (calendar['d'] >= 501  )  & ( calendar['d'] <= 510  ), 'NBAFinalsStart' ]  = 1
calendar.loc[ (calendar['d'] >= 860  )  & ( calendar['d'] <= 874  ), 'NBAFinalsStart' ]  = 1
calendar.loc[ (calendar['d'] >= 1224 )  & ( calendar['d'] <= 1234 ), 'NBAFinalsStart' ]  = 1
calendar.loc[ (calendar['d'] >= 1588 )  & ( calendar['d'] <= 1600 ), 'NBAFinalsStart' ]  = 1
calendar.loc[ (calendar['d'] >= 1952 )  & ( calendar['d'] <= 1969 ), 'NBAFinalsStart' ]  = 1

calendar.loc[ (calendar['d'] >= 123  )  & ( calendar['d'] <= 135  ), 'Sporting' ]  = 1
calendar.loc[ (calendar['d'] >= 501  )  & ( calendar['d'] <= 510  ), 'Sporting' ]  = 1
calendar.loc[ (calendar['d'] >= 860  )  & ( calendar['d'] <= 874  ), 'Sporting' ]  = 1
calendar.loc[ (calendar['d'] >= 1224 )  & ( calendar['d'] <= 1234 ), 'Sporting' ]  = 1
calendar.loc[ (calendar['d'] >= 1588 )  & ( calendar['d'] <= 1600 ), 'Sporting' ]  = 1
calendar.loc[ (calendar['d'] >= 1952 )  & ( calendar['d'] <= 1969 ), 'Sporting' ]  = 1

calendar.drop(['NBAFinalsEnd'], axis=1, inplace=True)

In [19]:
# Merging calendar 
with timer():
    train = train.merge(calendar, on='d', how='left')

print(train.shape)
print('-'*25)
train.tail()

Runtime: 23.42 sn
(60034810, 45)
-------------------------


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,month,year,Chanukah End,Christmas,Cinco De Mayo,ColumbusDay,Easter,Eid al-Fitr,EidAlAdha,Father's day,Halloween,IndependenceDay,LaborDay,LentStart,LentWeek2,MartinLutherKingDay,MemorialDay,Mother's day,NBAFinalsStart,NewYear,OrthodoxChristmas,OrthodoxEaster,Pesach End,PresidentsDay,Purim End,Ramadan starts,StPatricksDay,SuperBowl,Thanksgiving,ValentinesDay,VeteransDay,Cultural,National,Religious,Sporting
60034805,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1965,NaN,11620,5,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1
60034806,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1966,NaN,11620,6,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1
60034807,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1967,NaN,11620,7,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1
60034808,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1968,NaN,11621,1,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1
60034809,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1969,NaN,11621,2,6,2016,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1


In [20]:
del calendar; gc.collect()

40

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60034810 entries, 0 to 60034809
Data columns (total 45 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   object 
 1   item_id              object 
 2   dept_id              object 
 3   cat_id               object 
 4   store_id             object 
 5   state_id             object 
 6   d                    int16  
 7   sales                float64
 8   wm_yr_wk             int16  
 9   wday                 int8   
 10  month                int8   
 11  year                 int16  
 12  Chanukah End         int8   
 13  Christmas            int8   
 14  Cinco De Mayo        int8   
 15  ColumbusDay          int8   
 16  Easter               int8   
 17  Eid al-Fitr          int8   
 18  EidAlAdha            int8   
 19  Father's day         int8   
 20  Halloween            int8   
 21  IndependenceDay      int8   
 22  LaborDay             int8   
 23  LentStart            int8   
 

### 2.3. PRICE DATA PREPARATION

In [22]:
print(prices.shape)
print('-'*25)
prices.head()

(6841121, 4)
-------------------------


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [23]:
prices['sell_price_over_mean']   = (prices['sell_price'] / prices.groupby(['store_id', 'item_id'])['sell_price'].transform('mean')   ) - 1
prices['sell_price_over_median'] = (prices['sell_price'] / prices.groupby(['store_id', 'item_id'])['sell_price'].transform('median') ) - 1

prices['sell_price_over_mean']   = prices['sell_price_over_mean'  ].astype('float16')
prices['sell_price_over_median'] = prices['sell_price_over_median'].astype('float16')

print(prices.shape)
print('-'*25)
prices.head()

(6841121, 6)
-------------------------


,store_id,item_id,wm_yr_wk,sell_price,sell_price_over_mean,sell_price_over_median
0,CA_1,HOBBIES_1_001,11325,9.58,0.156250,0.15979
1,CA_1,HOBBIES_1_001,11326,9.58,0.156250,0.15979
2,CA_1,HOBBIES_1_001,11327,8.26,-0.003103,0.00000
3,CA_1,HOBBIES_1_001,11328,8.26,-0.003103,0.00000
4,CA_1,HOBBIES_1_001,11329,8.26,-0.003103,0.00000


In [24]:
# Merging prices 
with timer():
    train = train.merge(prices, on=['item_id','store_id','wm_yr_wk'], how='left')

print(train.shape)
print('-'*25)
train.tail()

Runtime: 36.62 sn
(60034810, 48)
-------------------------


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,month,year,Chanukah End,Christmas,Cinco De Mayo,ColumbusDay,Easter,Eid al-Fitr,EidAlAdha,Father's day,Halloween,IndependenceDay,LaborDay,LentStart,LentWeek2,MartinLutherKingDay,MemorialDay,Mother's day,NBAFinalsStart,NewYear,OrthodoxChristmas,OrthodoxEaster,Pesach End,PresidentsDay,Purim End,Ramadan starts,StPatricksDay,SuperBowl,Thanksgiving,ValentinesDay,VeteransDay,Cultural,National,Religious,Sporting,sell_price,sell_price_over_mean,sell_price_over_median
60034805,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1965,NaN,11620,5,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0
60034806,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1966,NaN,11620,6,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0
60034807,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1967,NaN,11620,7,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0
60034808,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1968,NaN,11621,1,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0
60034809,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1969,NaN,11621,2,6,2016,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,5.94,-0.0,0.0


In [25]:
del prices; gc.collect()

40

In [26]:
for col in ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']:
    train[col] = train[col].astype('category')

## 3 FEATURE ENGINEERING

In [27]:
## DO NOT FORGET TO ADD SNAP DATA
with timer():
    train = train.merge(snap, on=['d', 'state_id'], how='left')

print(train.shape)
print('-'*25)
train.tail()

Runtime: 17.84 sn
(60034810, 49)
-------------------------


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,month,year,Chanukah End,Christmas,Cinco De Mayo,ColumbusDay,Easter,Eid al-Fitr,EidAlAdha,Father's day,Halloween,IndependenceDay,LaborDay,LentStart,LentWeek2,MartinLutherKingDay,MemorialDay,Mother's day,NBAFinalsStart,NewYear,OrthodoxChristmas,OrthodoxEaster,Pesach End,PresidentsDay,Purim End,Ramadan starts,StPatricksDay,SuperBowl,Thanksgiving,ValentinesDay,VeteransDay,Cultural,National,Religious,Sporting,sell_price,sell_price_over_mean,sell_price_over_median,is_snap
60034805,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1965,NaN,11620,5,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,1
60034806,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1966,NaN,11620,6,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,0
60034807,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1967,NaN,11620,7,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,0
60034808,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1968,NaN,11621,1,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,0
60034809,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1969,NaN,11621,2,6,2016,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,5.94,-0.0,0.0,0


In [28]:
del snap; gc.collect()

20

In [29]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60034810 entries, 0 to 60034809
Data columns (total 49 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   id                      category
 1   item_id                 category
 2   dept_id                 category
 3   cat_id                  category
 4   store_id                category
 5   state_id                category
 6   d                       int16   
 7   sales                   float64 
 8   wm_yr_wk                int16   
 9   wday                    int8    
 10  month                   int8    
 11  year                    int16   
 12  Chanukah End            int8    
 13  Christmas               int8    
 14  Cinco De Mayo           int8    
 15  ColumbusDay             int8    
 16  Easter                  int8    
 17  Eid al-Fitr             int8    
 18  EidAlAdha               int8    
 19  Father's day            int8    
 20  Halloween               int8    
 21  Indepe

### Feature 1  >>>  id based statistics

In [30]:
f1 = train.groupby('id').agg({ 'sales': ['mean'] }).reset_index()
f1.columns = [ f[0] if f[1] == '' else 'id_' + f[0] + '_' + f[1] for f in f1.columns ]

print(f1.shape)
print('-'*25)
f1.tail()

(30490, 2)
-------------------------


,id,id_sales_mean
30485,HOUSEHOLD_2_516_TX_2,0.249871
30486,HOUSEHOLD_2_516_TX_3,0.125193
30487,HOUSEHOLD_2_516_WI_1,0.088099
30488,HOUSEHOLD_2_516_WI_2,0.032973
30489,HOUSEHOLD_2_516_WI_3,0.127254


In [31]:
# Merging f1
with timer():
    train = train.merge(f1, on='id', how='left')

del f1; gc.collect()

print(train.shape)
print('-'*25)
train.tail()

Runtime: 28.96 sn
(60034810, 50)
-------------------------


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,month,year,Chanukah End,Christmas,Cinco De Mayo,ColumbusDay,Easter,Eid al-Fitr,EidAlAdha,Father's day,Halloween,IndependenceDay,LaborDay,LentStart,LentWeek2,MartinLutherKingDay,MemorialDay,Mother's day,NBAFinalsStart,NewYear,OrthodoxChristmas,OrthodoxEaster,Pesach End,PresidentsDay,Purim End,Ramadan starts,StPatricksDay,SuperBowl,Thanksgiving,ValentinesDay,VeteransDay,Cultural,National,Religious,Sporting,sell_price,sell_price_over_mean,sell_price_over_median,is_snap,id_sales_mean
60034805,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1965,NaN,11620,5,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,1,0.127254
60034806,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1966,NaN,11620,6,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,0,0.127254
60034807,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1967,NaN,11620,7,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,0,0.127254
60034808,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1968,NaN,11621,1,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,0,0.127254
60034809,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1969,NaN,11621,2,6,2016,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,5.94,-0.0,0.0,0,0.127254


### Feature 2 >>> id-weekdaily based statistics

In [32]:
f2 = train.groupby(['id', 'wday']).agg({ 'sales': ['mean'] }).reset_index()
f2.columns = [ f[0] if f[1] == '' else 'id_wday_' +  f[0] + '_' + f[1] for f in f2.columns ]

print(f2.shape)
print('-'*25)
f2.tail()

(213430, 3)
-------------------------


,id,wday,id_wday_sales_mean
213425,HOUSEHOLD_2_516_WI_3,3,0.108303
213426,HOUSEHOLD_2_516_WI_3,4,0.083032
213427,HOUSEHOLD_2_516_WI_3,5,0.086643
213428,HOUSEHOLD_2_516_WI_3,6,0.133574
213429,HOUSEHOLD_2_516_WI_3,7,0.144404


In [33]:
# Merging f2
with timer():
    train = train.merge(f2, on=['id', 'wday'], how='left')

del f2; gc.collect()

print(train.shape)
print('-'*25)
train.tail()

Runtime: 30.43 sn
(60034810, 51)
-------------------------


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,month,year,Chanukah End,Christmas,Cinco De Mayo,ColumbusDay,Easter,Eid al-Fitr,EidAlAdha,Father's day,Halloween,IndependenceDay,LaborDay,LentStart,LentWeek2,MartinLutherKingDay,MemorialDay,Mother's day,NBAFinalsStart,NewYear,OrthodoxChristmas,OrthodoxEaster,Pesach End,PresidentsDay,Purim End,Ramadan starts,StPatricksDay,SuperBowl,Thanksgiving,ValentinesDay,VeteransDay,Cultural,National,Religious,Sporting,sell_price,sell_price_over_mean,sell_price_over_median,is_snap,id_sales_mean,id_wday_sales_mean
60034805,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1965,NaN,11620,5,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,1,0.127254,0.086643
60034806,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1966,NaN,11620,6,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,0,0.127254,0.133574
60034807,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1967,NaN,11620,7,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,0,0.127254,0.144404
60034808,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1968,NaN,11621,1,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,0,0.127254,0.161871
60034809,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1969,NaN,11621,2,6,2016,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,5.94,-0.0,0.0,0,0.127254,0.172662


### Feature 3 >>> id-monthly based statistics

In [34]:
f3 = train.groupby(['id', 'month']).agg({ 'sales': ['mean'] }).reset_index()
f3.columns = [ f[0] if f[1] == '' else 'id_month_' + f[0] + '_' + f[1] for f in f3.columns ]

print(f3.shape)
print('-'*25)
f3.tail()

(365880, 3)
-------------------------


,id,month,id_month_sales_mean
365875,HOUSEHOLD_2_516_WI_3,8,0.129032
365876,HOUSEHOLD_2_516_WI_3,9,0.113333
365877,HOUSEHOLD_2_516_WI_3,10,0.103226
365878,HOUSEHOLD_2_516_WI_3,11,0.133333
365879,HOUSEHOLD_2_516_WI_3,12,0.141935


In [35]:
# Merging f3
with timer():
    train = train.merge(f3, on=['id', 'month'], how='left')

del f3; gc.collect()

print(train.shape)
print('-'*25)
train.tail()

Runtime: 31.71 sn
(60034810, 52)
-------------------------


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,month,year,Chanukah End,Christmas,Cinco De Mayo,ColumbusDay,Easter,Eid al-Fitr,EidAlAdha,Father's day,Halloween,IndependenceDay,LaborDay,LentStart,LentWeek2,MartinLutherKingDay,MemorialDay,Mother's day,NBAFinalsStart,NewYear,OrthodoxChristmas,OrthodoxEaster,Pesach End,PresidentsDay,Purim End,Ramadan starts,StPatricksDay,SuperBowl,Thanksgiving,ValentinesDay,VeteransDay,Cultural,National,Religious,Sporting,sell_price,sell_price_over_mean,sell_price_over_median,is_snap,id_sales_mean,id_wday_sales_mean,id_month_sales_mean
60034805,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1965,NaN,11620,5,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,1,0.127254,0.086643,0.093333
60034806,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1966,NaN,11620,6,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,0,0.127254,0.133574,0.093333
60034807,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1967,NaN,11620,7,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,0,0.127254,0.144404,0.093333
60034808,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1968,NaN,11621,1,6,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,5.94,-0.0,0.0,0,0.127254,0.161871,0.093333
60034809,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1969,NaN,11621,2,6,2016,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,5.94,-0.0,0.0,0,0.127254,0.172662,0.093333


In [36]:
cols_to_drop = ['id', 'cat_id', 'state_id',
                'Chanukah End', 'Christmas', 'Cinco De Mayo', 'ColumbusDay', 'Easter', 
                'Halloween', 'IndependenceDay', 'LaborDay', 'LentStart', 'LentWeek2', 'MartinLutherKingDay',
                'MemorialDay', 'NewYear', 'OrthodoxChristmas', 'OrthodoxEaster', 'Pesach End',
                'PresidentsDay', 'Purim End', 'StPatricksDay', 'Thanksgiving', 
                'ValentinesDay', 'VeteransDay']
train.drop(cols_to_drop, axis=1, inplace=True)

print(train.shape)
print('-'*25)
train.tail()

(60034810, 27)
-------------------------


,item_id,dept_id,store_id,d,sales,wm_yr_wk,wday,month,year,Eid al-Fitr,EidAlAdha,Father's day,Mother's day,NBAFinalsStart,Ramadan starts,SuperBowl,Cultural,National,Religious,Sporting,sell_price,sell_price_over_mean,sell_price_over_median,is_snap,id_sales_mean,id_wday_sales_mean,id_month_sales_mean
60034805,HOUSEHOLD_2_516,HOUSEHOLD_2,WI_3,1965,NaN,11620,5,6,2016,0,0,0,0,1,1,0,0,0,1,1,5.94,-0.0,0.0,1,0.127254,0.086643,0.093333
60034806,HOUSEHOLD_2_516,HOUSEHOLD_2,WI_3,1966,NaN,11620,6,6,2016,0,0,0,0,1,1,0,0,0,1,1,5.94,-0.0,0.0,0,0.127254,0.133574,0.093333
60034807,HOUSEHOLD_2_516,HOUSEHOLD_2,WI_3,1967,NaN,11620,7,6,2016,0,0,0,0,1,1,0,0,0,1,1,5.94,-0.0,0.0,0,0.127254,0.144404,0.093333
60034808,HOUSEHOLD_2_516,HOUSEHOLD_2,WI_3,1968,NaN,11621,1,6,2016,0,0,0,0,1,1,0,0,0,1,1,5.94,-0.0,0.0,0,0.127254,0.161871,0.093333
60034809,HOUSEHOLD_2_516,HOUSEHOLD_2,WI_3,1969,NaN,11621,2,6,2016,0,0,1,0,1,1,0,1,0,1,1,5.94,-0.0,0.0,0,0.127254,0.172662,0.093333


## 4. MODELLING

In [37]:
pred = train[train['d'] > 1913]
y_true = pred['sales']
pred = pred.drop('sales', axis=1)

valid = train[ (train['d'] <= 1941) & (train['d'] > 1913) ]
y_valid = valid['sales']
x_valid = valid.drop('sales', axis=1)

train = train[train['d'] < 1913]
x_train = train.drop('sales', axis=1)
y_train = train['sales']

In [38]:
cat_cols = ['item_id', 'dept_id', 'store_id' ]
feature_columns_to_keep = x_train.columns
out_path = 'output/'

In [39]:
# pred.to_csv(f'{out_path}model_lgbm_1_pred.csv', index=False)
# y_true.to_csv(f'{out_path}model_lgbm_1_y_true.csv', index=False)

In [40]:
train_data = lgb.Dataset( x_train, label = y_train,  categorical_feature = cat_cols, free_raw_data=False )
valid_data = lgb.Dataset( x_valid, label = y_valid,  categorical_feature = cat_cols, free_raw_data=False )

In [41]:
del  train, valid, x_train, x_valid, y_train, y_valid ; gc.collect()

80

In [ ]:
feature_importances = pd.DataFrame()
feature_importances['feature'] = feature_columns_to_keep


obj_list   = [ 'regression' ]
boost_list = [ 'gbdt']
lr_list    = [ 0.1, 0.3, 0.7 ]
l1_list    = [ 0, 0.3, 0.7 ]
l2_list    = [ 0, 0.3, 0.7 ]

res_rmse = pd.DataFrame()

i=0


for obj in obj_list:
    for boost in boost_list:
        for lr in lr_list: 
            for l1 in l1_list:
                for l2 in l2_list:
                    i += 1
                    param_name = f'{i}_{obj}_{boost}_{str(lr)}_{str(l1)}_{str(l2)}'
                    
                    params = {
                    # 'regression', 'regression_l1', 'tweedie',  'poisson', 'quantile', 'gamma', 'multiclass', 'cross_entropy'
                    'objective'         : obj        ,
                        
                    #
                    # 'num_class'         : len(set(train_data.label)) ,

                    # 'gbdt', 'rf', 'dart', 'goss'
                    'boosting'          : boost      ,

                    # number of boosting iterations (100)  - [0, )
                    'num_iterations'    : 250        ,    

                    # shrinkage rate (0.1) - [0, )
                    'learning_rate'     : lr         ,

                    # max number of leaves in one tree (31) - [1,131072]
                    'num_leaves'        : 128        ,    

                    # 'serial', 'feature', 'data', 'voting'
                    'tree_learner'      : 'serial'   ,

                    # if there are too many rows make it True, if there are too many columns make 'force_col_wise' option True
                    # Never make them both True
                    'force_row_wise'    : True       ,

                    # if the data is small use it otherwise don't touch it
                    'max_depth'         : -1         ,

                    # 20 - [0, )
                    'min_data_in_leaf'  : 50         , 

                    # randomly select part of data without resampling, to enable bagging it must be smaller than 1 (0,1]
                    # 'bagging_fraction'  : 1          ,

                    # to enable bagging bagging_freq must be nonzero [0, )
                    #'bagging_freq'      : 0          ,

                    # randomly select features, to enable featuring it must be smaller than 1 (0,1]
                    # 'feature_fraction'  : 1          ,

                    # stops training if one metric of one validation data does not improve in the last x rounds (0)
                    'early_stopping_round' : 250     ,

                    # (0)
                    'lambda_l1'         : l1         ,

                    # (0)
                    'lambda_l2'         : l2          ,

                    #controls the level of LGBM verbosity
                    'verbosity'         : 1          ,  

                    # 'rmse', 'auc', 'l1', 'l2', 'tweedie', 'poissson', 'multi_logloss'..
                    'metric'            : 'rmse'      ,
                        
                    'random_state': 28,

                    }

                    with timer():
                        model = lgb.train(params, train_data, 2000,  valid_sets = [valid_data], verbose_eval=20 ) 
                        model.save_model(f'{path}model_lgbm_1_' + param_name + '.txt')

                    feature_importances[param_name] = model.feature_importance()               

                    plt.figure( figsize=(12,8) )
                    sns.barplot( x    = param_name,
                                 y    = 'feature',
                                 data = feature_importances.sort_values(param_name, ascending = False ) )
                    plt.title('LGBM Feature Importance ' + param_name)
                    plt.show()

                    res_rmse = res_rmse.append( pd.DataFrame( {  'i': [i],
                                                                 'objective': [obj],
                                                                 'boosting': [boost],
                                                                 'learning_rate':[lr], 
                                                                 'l1':[l1], 
                                                                 'l2':[l2], 
                                                                 'RMSE': [model.best_score['valid_0']['rmse'] ]  } ) )

                    print('-'*25)
                    print( res_rmse )
                    print('-'*25)

                
res_rmse.to_excel(f'{out_path}res_rmse.xlsx', index=False)                 
feature_importances.to_excel(f'{out_path}feature_importances.xlsx', index=False) 

In [ ]:
# pred   = pd.read_csv(f'{path}model_lgbm_1_pred.csv')
# y_true = pd.read_csv(f'{path}model_lgbm_1_y_true.csv')

In [2]:
# res_rmse = pd.read_excel(f'{out_path}res_rmse.xlsx')                 
# feature_importances = pd.read_excel(f'{out_path}feature_importances.xlsx') 

In [ ]:
# res_rmse

In [ ]:
# model = lgb.Booster(model_file='model.txt')

In [ ]:
y_pred = model.predict(pred)

In [ ]:
pred['y_true'] = y_true
pred['y_pred'] = y_pred
pred.head()

In [ ]:
pred['id'] = pred['item_id'].astype(str) + '_' + pred['store_id'].astype(str)

In [ ]:
res_val  = pred[pred['d'] <= 1941][['id', 'd', 'y_pred']].copy()
res_eval = pred[pred['d'] > 1941][['id', 'd', 'y_pred']].copy() 

res_val['id']  = res_val['id']  + '_validation'
res_eval['id'] = res_eval['id'] + '_evaluation'

res_val             = res_val.pivot_table(index='id', columns='d', values='y_pred').reset_index()
res_eval            = res_eval.pivot_table(index='id', columns='d', values='y_pred').reset_index()
res_eval.columns    = res_val.columns

res            = pd.concat([res_val, res_eval], axis=0, sort=False, ignore_index=True)
res            = submission[['id']].merge(res, on='id', how='left')
res.columns    = submission.columns
res.head()

In [ ]:
res.shape

In [ ]:
res.to_csv(f'{out_path}M5_Forecasting_Model_Raw_LGBM.csv', index=False)

In [ ]:
from sklearn.metrics import mean_squared_error
valid = pred[pred['d'] <= 1941]
print('LGBM Raw RMSE       :   ', mean_squared_error(valid['y_true'], valid['y_pred'] )**(0.5)   )
print('LGBM Raw Round RMSE :   ', mean_squared_error(valid['y_true'], round(valid['y_pred'], 0) )**(0.5)   )

In [ ]:
sns.lineplot(x='d', y='y_true', data=pred[ ( pred['item_id'] == 'FOODS_3_013' ) & ( pred['store_id'] == 'CA_2' ) ] , color='r')
sns.lineplot(x='d', y='y_pred', data=pred[ ( pred['item_id'] == 'FOODS_3_013' ) & ( pred['store_id'] == 'CA_2' ) ], color='b')
plt.show()

In [ ]:
sns.lineplot(x='d', y='y_true', data=pred[ ( pred['item_id'] == 'FOODS_2_102' ) & ( pred['store_id'] == 'WI_2' ) ], color='r')
sns.lineplot(x='d', y='y_pred', data=pred[ ( pred['item_id'] == 'FOODS_2_102' ) & ( pred['store_id'] == 'WI_2' ) ], color='b')
plt.show()